# Deep Deterministic Policy Gradient

На этом семинаре мы будем обучать нейронную сеть на фреймворке __pytorch__ с помощью алгоритма Deep Deterministic Policy Gradient.

## Теория

Deep Deterministic Policy Gradient (DDPG) - это алгоритм, который одновременно учит Q-функцию и стратегию. Он использует off-policy данные и уравнения Беллмана для обучения Q-функции, а Q-функция используется для обучения стратегии.

Данный подход тесно связан с Q-обучением и мотивирован следующей идеей: если вы знаете оптимальную функцию action-value $Q^*(s,a)$, тогда для конкретного состояния, оптимальное действие $a^*(s)$ может быть найдено решением:

$$a^*(s) = \arg \max_a Q^*(s,a).$$

Для сред с дискретным пространством действий - это легко, вычисляем полезности для каждого из действий, а потом берем максимум. Для непрерывных действий - это сложная оптимизационная задача.

DDPG чередует обучение аппроксиматора $Q^*(s,a)$ с обучением аппроксиматора  $a^*(s)$, и делает это специальным образом именно для непрерывных (continuous) сред, что отражается в том как алгоритм вычисляет $\max_a Q^*(s,a)$.
Поскольку пространство действий непрерывно, предполагается, что функция $Q^*(s,a)$ дифференцируема по аргументу действия. Это позволяет нам установить эффективное правило обучения на основе градиента для стратегии $\mu(s)$.

<img src="https://spinningup.openai.com/en/latest/_images/math/5811066e89799e65be299ec407846103fcf1f746.svg">

Оригинальная статья:  <a href="https://arxiv.org/abs/1509.02971">Continuous control with deep reinforcement learning Arxiv</a>

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

### Вспомогательные функции

In [ ]:
def print_mean_reward(step, session_rewards, eval_session_rewards = None):
    if not session_rewards:
        return

    def get_mean_reward(rewards):
        return round(sum(rewards) / len(rewards), 2)

    train_mean = get_mean_reward(session_rewards)
    eval_mean = None
    if eval_session_rewards is not None:
        eval_mean = get_mean_reward(eval_session_rewards)

    print(f"step: {str(step).zfill(6)}, train: {train_mean}, eval: {eval_mean}")
    return train_mean if eval_mean is None else eval_mean


def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x).to(device)
    return x

## Batch/replay buffer


In [ ]:
from collections import deque, namedtuple

Transition = namedtuple('Transition', ['state', 'action', 'reward', 'next_state', 'done'])


class ReplayBuffer:
    def __init__(self, size):
        self.buffer = deque(maxlen=size)
        self.rng = np.random.default_rng()

    def append(self, state, action, reward, next_state, done):
        sample = Transition(state, action, reward, next_state, done)
        self.buffer.append(sample)

    def sample_batch(self, batch_size):
        indices = self.rng.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, next_states, dones = [], [], [], [], []
        for i in indices:
            s, a, r, n_s, done = self.buffer[i]
            states.append(s)
            actions.append(a)
            rewards.append(r)
            next_states.append(n_s)
            dones.append(done)

        batch = np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)
        return batch, indices

    def __len__(self):
        return len(self.buffer)

## Исследование - GaussNoise
Добавляем Гауссовский шум к действиям детерминированной стратегии.
Добавляем его только при обучении для исследования.

NB: вы также можете погуглить реализации шума из распределения Орнштейна-Уленбека (Ornstein-Uhlenbeck) — это даст агенту более качественное направленное исследование.

In [ ]:
class GaussNoise:
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def sample(self, action):
        return np.random.normal(action, self.sigma)

## DDPG Network

<img src="https://raw.githubusercontent.com/Tviskaron/mipt/master/2020/RL/figures/DQN-DDPG.svg">

### DDPG Model
Реализуйте модель актор-критика `DdpgModel`. Можете реализовать актор-критика единым модулем, а можете разнести их (первый вариант не обязательно предполагает общее тело сетей). `DdpgModel` также не обязательно делать наследником `nn.Module`.

NB: часто рекомендуется инициализировать последний слой актора весами с небольшими по модулю значениями ($\sim 10^{-3}$)

In [ ]:
class DdpgModel:
    def __init__(self, state_dim, hidden_dim, action_dim):
        super().__init__()

        self._policy = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, action_dim),
            nn.Tanh(),
        ).to(device)

        init_w = 1e-3
        self._policy[-2].weight.data.uniform_(-init_w, init_w)

        self._q = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        ).to(device)

    def act(self, s):
        s = to_tensor(s)
        ####### Здесь ваш код ########
        return ...
        ##############################

    def Q(self, s, a):
        s, a = to_tensor(s), to_tensor(a)
        sa = torch.cat([s, a], 1)
        ####### Здесь ваш код ########
        return ...
        ##############################

    def V(self, s):
        s = to_tensor(s)
        ####### Здесь ваш код ########
        return ...
        ##############################

    def copy_params_to(self, model: 'DdpgModel', lr=1.0):
        self.alpha_merge_params(target=model._policy, other=self._policy, lr=lr)
        self.alpha_merge_params(target=model._q, other=self._q, lr=lr)

    @staticmethod
    def alpha_merge_params(target: nn.Module, other: nn.Module, lr):
        state_dict, other_dict = target.state_dict(), other.state_dict()
        target.load_state_dict({
            key: (1 - lr) * state_dict[key] + lr * other_dict[key]
            for key in state_dict
        })

### DDPG Agent

Реализуйте класс агента `DdpgAgent`, который содержит:

- обучаемую модель актор-критика и его оптимизаторы (например, `torch.optim.Adam`)
- периодически обновляемую копию модели с замороженными весами для вычисления TD target.
- метод `act` для выбора действия. Можете добавить флаг `learn` для подмешивания гауссовского шума при обучении, либо добавляйте его снаружи агента.
- метод `learn` для обновления модели агента (и актора, и критика) на новом пакете опыта.

In [ ]:
class DdpgAgent:
    def __init__(self, state_dim, action_dim, hidden_dim, lr, gamma, soft_tau, noise_sigma, replay_buffer_size):
        self.lr = lr
        self.gamma = gamma
        self.soft_tau = soft_tau
        self.noise = GaussNoise(sigma=noise_sigma)

        # Инициализируйте модель актор-критика и SGD оптимизатор (например, `torch.optim.Adam)`)
        self.model = DdpgModel(state_dim, hidden_dim, action_dim)
        self.actor_optim = torch.optim.Adam(self.model._policy.parameters(), lr=0.3 * lr)
        self.critic_optim = torch.optim.Adam(self.model._q.parameters(), lr=lr)

        self.target_model = DdpgModel(state_dim, hidden_dim, action_dim)
        self.model.copy_params_to(self.target_model)

        self.replay_buffer = ReplayBuffer(replay_buffer_size)

    def act(self, state, *, learn):
        with torch.no_grad():
            action = self.model.act(state).cpu().numpy()

        if learn:
            action = self.noise.sample(action)
        return action

    def update(self, batch_size, sgd_steps_per_update):
        if len(self.replay_buffer) < batch_size:
            return

        # ограничивает сверху количество эпох для буфера небольшого размера
        sgd_steps_per_update = min(sgd_steps_per_update, 2 * len(self.replay_buffer) // batch_size)
        for _ in range(sgd_steps_per_update):
            train_batch, indices = self.replay_buffer.sample_batch(batch_size)
            states, actions, rewards, next_states, is_done = train_batch

            states = to_tensor(states)                             # shape: [batch_size, state_size]
            actions = to_tensor(actions)                           # shape: [batch_size, 1]
            rewards = to_tensor(rewards).unsqueeze(1)              # shape: [batch_size, 1]
            next_states = to_tensor(next_states)                   # shape: [batch_size, state_size]
            is_done = to_tensor(is_done, bool).unsqueeze(1)        # shape: [batch_size, 1]

            self.update_actor(states)
            self.update_critic(states, actions, rewards, next_states, is_done)

        # update target model
        self.model.copy_params_to(self.target_model, lr=self.soft_tau)

    def update_actor(self, states):
        # loss = ...
        ####### Здесь ваш код ########
        loss = ...
        ##############################
        self.actor_optim.zero_grad()
        loss.backward()
        self.actor_optim.step()

    def update_critic(self, states, actions, rewards, next_states, is_done):
        q_values = self.model.Q(states, actions)
        with torch.no_grad():
            V_next = self.target_model.V(next_states)
            td_target = rewards + self.gamma * torch.logical_not(is_done) * V_next

        # loss = ...
        ####### Здесь ваш код ########
        loss = ...
        ##############################

        self.critic_optim.zero_grad()
        loss.backward()
        self.critic_optim.step()

## Evaluation

Реализуйте функцию `run`, которая принимает среду, гиперпараметры агента и условие останова эксперимента (return threshold $G_{target}$). Используйте функцию `print_mean_reward` для вывода промежуточных результатов качества агента в трейн и eval режимах.

Проведите эксперимент на среде с непрерывным пространством действий (например, continuous montain car или pendulum).

In [ ]:
from gymnasium.experimental.wrappers import RescaleActionV0

def run_episode(
        env: gym.Env, agent: DdpgAgent, step: int, train: bool,
        steps_per_update: int, batch_size: int, update_iterations: int
):
    episode_reward = 0
    done = False
    state, _ = env.reset()

    while not done:
        step += 1

        action = agent.act(state, learn=train)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        if train:
            agent.replay_buffer.append(state, action, reward, next_state, done)
            if step % steps_per_update == 0:
                agent.update(batch_size, update_iterations)

        state = next_state
        episode_reward += reward

    return step, episode_reward


def run(
        env: gym.Env, hidden_size: int, lr: float, gamma: float, soft_tau: float, max_episodes: int,
        noise_sigma: float, replay_buffer_size: int,
        steps_per_update: int, batch_size: int, update_iterations: int,
        success_reward: float
):
    env = RescaleActionV0(env, min_action=-1.0, max_action=1.0)

    states_dim = env.observation_space.shape[0]
    actions_dim = env.action_space.shape[0]
    agent = DdpgAgent(states_dim, actions_dim, hidden_size, lr, gamma, soft_tau, noise_sigma, replay_buffer_size)

    step = 0
    train_results, eval_results = [], []
    for i_episode in range(1, max_episodes + 1):
        _, episode_reward = run_episode(
            env, agent, step, train=False,
            steps_per_update=steps_per_update, batch_size=batch_size, update_iterations=update_iterations
        )
        eval_results.append(episode_reward)

        step, episode_reward = run_episode(
            env, agent, step, train=True,
            steps_per_update=steps_per_update, batch_size=batch_size, update_iterations=update_iterations
        )
        train_results.append(episode_reward)

        if i_episode % 5 == 0 and print_mean_reward(step, train_results[-10:], eval_results[-10:]) >= success_reward:
            print('Принято!')
            return

# env, success = gym.make("MountainCarContinuous-v0", max_episode_steps=1000), 95.0
env, success = gym.make("Pendulum-v1", max_episode_steps=200), -200

# run experiment
run(
    env = env,
    max_episodes = 100,  # количество эпизодов обучения
    hidden_size = 128,  # кол-во переменных в скрытых слоях
    lr = 0.001, # learning rate
    gamma = 0.995,  # дисконтирующий множитель,
    soft_tau = 0.01, # скорость обновления target сети
    replay_buffer_size = 5000,
    noise_sigma = 0.15,
    steps_per_update = 10,  # через столько шагов стратегия будет обновляться
    batch_size = 100,
    update_iterations = 4,
    success_reward=success
)